In [ ]:
import  numpy as np
import pandas as pd
import json
from google.colab import drive
drive.mount('/content/drive')

#setting tpu config
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

MessageError: ignored

In [ ]:
# !pip install tensorflow-text
!pip install -U "tensorflow==2.9.*"
!pip install -q -U "tensorflow-text==2.9.*"
import tensorflow as tf
import tensorflow_text as tf_text

In [ ]:
df=pd.DataFrame(columns=["Key","Value"])
df.columns

In [ ]:
import json
from sklearn.preprocessing import LabelBinarizer

with open('/content/drive/MyDrive/train.json', 'r') as f:
    train = json.load(f)

y = list(train.values())
x = list(train.keys())

In [ ]:
df["Key"]=pd.Series(x[:4152659])
df["Value"]=pd.Series(y[:4152659])

In [ ]:
df.tail(5)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
df['Value'].value_counts()

In [ ]:
df_normal=df[df["Value"]=="normal"]
df_abnormal=df[df["Value"]=="abnormal"]

In [ ]:
df_normal.shape,df_abnormal.shape

In [ ]:
df_abnormal_upscaled=df_abnormal.sample(df_normal.shape[0],replace=True)

In [ ]:
df_abnormal_upscaled.shape

In [ ]:
df_balanced = pd.concat([df_normal, df_abnormal_upscaled])
df_balanced.shape

In [ ]:
df_balanced['Value'].value_counts()

In [ ]:
df_balanced['Category']=df_balanced['Value'].apply(lambda x: 1 if x=='normal' else 0)
df_balanced.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_balanced['Key'],df_balanced['Category'], stratify=df_balanced['Category'])

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']



In [ ]:
p=get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)
print(p)

In [ ]:
#BUILDING MODEL
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
history=model.fit(x_train, y_train,validation_split=0.3, epochs=10)